In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# **Tabla de contenidos**
***
Redes Neuronales (MLP)
1.  Modelo 1 (baseline)

    1.1. Creación del modelo baseline

    1.2. Evaluación del modelo baseline

2.  Modelo 2

    2.1. Creación del modelo 2

    2.2. Evaluación del modelo 2
    
3.  Modelo 3

    3.1. Creación del modelo 3

    3.2. Evaluación del modelo 3

4.  Evaluación general y conclusiones

***

# **Redes Neuronales** (MLP)

Un **modelo MLP** *(perceptrón multicapa)* es una clase de red neuronal que se caracteriza por estar inspirado en las neuronas biológicas y su principal cometido es la **clasificación de datos**.

En nuestro caso vamos a utilizar un modelo MLP para predecir y clasificar **qué tipo de crimen es más posible que le ocurra a una víctima** dependiendo de los siguientes datos:

In [6]:
X_train = pd.read_csv("../../../data/x_train.csv")
X_test = pd.read_csv("../../../data/x_test.csv")
y_train = pd.read_csv("../../../data/y_train.csv")
y_test = pd.read_csv("../../../data/y_test.csv")

Nuestras **variables predictoras** son:

In [11]:
print(f"Variables predictoras X_train: {X_train.columns.tolist()}")
print(f"Variables predictoras X_test: {X_test.columns.tolist()}")

Variables predictoras X_train: ['ager_18-24', 'ager_25-34', 'ager_35-49', 'ager_50-64', 'ager_65+', 'sex_Male', 'race_ethnicity_Non-Hispanic American Indian/Alaska Native', 'race_ethnicity_Non-Hispanic Asian/Native Hawaiian/Other Pacific Islander', 'race_ethnicity_Non-Hispanic black', 'race_ethnicity_Non-Hispanic more than one race', 'race_ethnicity_Non-Hispanic white', 'hincome1_$25,000 to $34,999', 'hincome1_$35,000 to $49,999', 'hincome1_$50,000 to $74,999', 'hincome1_$7,500 to $14,999', 'hincome1_$75,000 or more', 'hincome1_Less than $7,500', 'hincome1_Residue', 'marital_Married', 'marital_Never married', 'marital_Residue', 'marital_Separated', 'marital_Widowed', 'popsize_100,000–249,999', 'popsize_250,000–499,999', 'popsize_500,000–999,999', 'popsize_Not a place', 'popsize_Under 100,000', 'region_Northeast', 'region_South', 'region_West', 'msa_Outside MSA', 'msa_Principal city within MSA', 'educatn1_Grade school', 'educatn1_High school', 'educatn1_Middle school', 'educatn1_No scho

En este caso la **variable a predecir** seria *'newoff'*:

In [9]:
print(f"Variable a predecir y_train: {y_train.columns[0]}")
print(f"Variable a predecir y_test: {y_test.columns[0]}")

Variable a predecir y_train: newoff
Variable a predecir y_test: newoff


Nuestros DataFrames tiene la siguiente **dimensionalidad**:

In [12]:
print("Dimensionalidad de X_train:", X_train.shape)
print("Dimensionalidad de X_test:", X_test.shape)
print("Dimensionalidad de y_train:", y_train.shape)
print("Dimensionalidad de y_test:", y_test.shape)

Dimensionalidad de X_train: (44702, 47)
Dimensionalidad de X_test: (11176, 47)
Dimensionalidad de y_train: (44702, 1)
Dimensionalidad de y_test: (11176, 1)


Teniendo esto en cuenta podemos hacer las siguientes presunciones:

1. La dimensionalidad de la capa de entrada siempre va a ser:  
$$
n_{\text{features}} = X_{\text{train}}.shape[1]
$$
2. Las capas ocultas utilizarán la función de activación *'ReLU'*:
$$
\text{ReLU}(x) = \max(0, x)
$$
3. Como estamos haciendo una red neuronal para clasificación multiclase la función de activación de la capa de salida sera *'Softmax'*:
$$
\text{Softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}, \quad i = 1, \dots, K
$$
4. La función de pérdida utilizada será la entropía cruzada categórica (*categorical cross-entropy*):
$$
\mathcal{L} = - \sum_{i=1}^{K} y_i \log(\hat{y}_i)


## 1. Explicación modelo baseline

### 1.1. Creación modelo baseline


0        5
1        2
2        4
3        4
4        4
        ..
68847    4
68848    4
68849    4
68850    4
68851    4
Name: newoff, Length: 68852, dtype: int64

In [ ]:
# Modelo simple
model_baseline = Sequential([
    Dense(32, activation='relu', input_shape=(16,)),
    Dense(7, activation='softmax')  # 7 clases
])

In [ ]:
model_baseline.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history_crime = model_baseline.fit(X_train_scaled, y_train,
                                   validation_split=0.2,
                                   epochs=20,
                                   batch_size=32,
                                   verbose=1)

### 1.2. Evaluación modelo baseline

In [ ]:
# Evaluar train

train_loss, train_acc = model_baseline.evaluate(X_train_scaled, y_train, verbose=0)

print(f"Train Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Train Loss: {train_loss:.4f}")
print("="*50)

print("\n Baseline del azar (3 clases): 33.3%")

In [ ]:
# Evaluar test

test_loss, test_acc = model_baseline.evaluate(X_test_scaled, y_test, verbose=0)

print("="*50)
print(" RESULTADOS BASELINE - Tipos crimen")
print("="*50)
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print("="*50)

In [ ]:
# Visualización de curvas de entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Crear rango de epochs (1 a 20, no 0 a 19)
epochs = range(1, len(history_crime.history['loss']) + 1)

# Loss
ax1.plot(epochs, history_crime.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(epochs, history_crime.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Curvas de Loss')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(epochs, history_crime.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(epochs, history_crime.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Curvas de Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de confusión
y_pred_crime = model_baseline.predict(X_test_scaled)
y_pred_crime_classes = np.argmax(y_pred_crime, axis=1)

cm = confusion_matrix(y_test, y_pred_crime_classes)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay'],
            yticklabels=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay'])
plt.title('Matriz de Confusión - Tipos judía')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\n Classification Report:")
print(classification_report(y_test, y_pred_crime_classes, 
                          target_names=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay']))

## 2. Explicación modelo 2

### 2.1. Creación modelo 2

### 2.2. Evaluación modelo 2

## 3. Explicación modelo 3

### 3.1. Creación modelo 3

### 3.2. Evaluación modelo 3

# Evaluación general de todos los modelos